# Im aiworkshop importieren

In [ ]:
from __future__ import annotations
from aiworkshop_utils.standardlib_imports import os, json, base64, Optional, List, Dict, Any, Union, pprint, asyncio, uuid
from aiworkshop_utils.thirdparty_imports import requests, BaseModel, Field, DataType, MilvusClient, Collection, rprint
from aiworkshop_utils.custom_utils import show_pretty_json, encode_image
from aiworkshop_utils.jupyter_imports import display
from aiworkshop_utils.docling_imports import HybridChunker, DocumentConverter
from aiworkshop_utils.openai_imports import OpenAI, Agent, Runner, OpenAIChatCompletionsModel, AsyncOpenAI, set_tracing_disabled, ModelSettings, function_tool, trace, RunContextWrapper
from aiworkshop_utils import config
from aiworkshop_utils import embedders

# Außerhalb aiworkshop importieren

## UV Dependencies

```
dependencies = [
  "ipykernel",
  "jupyterlab",
  "numpy",
  "pandas",
  "matplotlib",
  "scikit-learn",  # for cosine_similarity
  "requests",
  "pymilvus",
  "openai",
  "openai-agents",  # for the Agent, Runner classes
  "pydantic",  # for BaseModel
  "python-dotenv",
  "docling",  # for document processing
  "tqdm",
  "rich",
  "duckduckgo-search"
]
```

## Direkte Imports

In [ ]:
# Standard library imports
import os
import json
import base64
import logging
from typing import Optional, List, Dict, Any, Union, Literal
import asyncio
from datetime import datetime, date, time, timezone
from zoneinfo import ZoneInfo
import uuid
from dataclasses import dataclass
from pprint import pprint
from glob import glob

# Third-party imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
import requests
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from rich import print as rprint
from tqdm import tqdm
from pymilvus import DataType, MilvusClient, Collection
from duckduckgo_search import DDGS

# Document processing imports
from docling.backend.pypdfium2_backend import PyPdfiumDocumentBackend
from docling.chunking import HybridChunker
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import (
    AcceleratorDevice,
    AcceleratorOptions,
    PdfPipelineOptions,
)
from docling.document_converter import (
    DocumentConverter,
    PdfFormatOption,
    WordFormatOption,
)
from docling.pipeline.simple_pipeline import SimplePipeline

# OpenAI and Agent imports
from openai import OpenAI, AsyncOpenAI
from agents import (
    Agent,
    GuardrailFunctionOutput,
    InputGuardrail,
    InputGuardrailTripwireTriggered,
    ModelSettings,
    OpenAIChatCompletionsModel,
    Runner,
    function_tool,
    set_tracing_disabled,
    trace,
    RawResponsesStreamEvent,
    ItemHelpers, 
    MessageOutputItem,
    RunContextWrapper,
    input_guardrail,
    output_guardrail
)
from openai.types.responses import ResponseContentPartDeltaEvent, ResponseTextDeltaEvent

# Pipeline

## [Parsing -> Chunking -> Embedding -> Indexing] Pipeline

In [2]:
class DocumentParser:
    def __init__(self, converter):
        """
        :param converter: An instance of DocumentConverter configured with necessary options.
        """
        self.converter = converter

    def parse(self, file_path: str, options: dict = None):
        """
        Convert the document at file_path into a DoclingDocument.
        :param file_path: Path to the document.
        :param options: Optional processing options.
        :return: DoclingDocument
        """
        # You can merge options with default settings if needed.
        print(f"Converting document: {file_path}")
        result = self.converter.convert(file_path)
        return result.document

In [3]:
source = "assets/doc_storage/ECTS_Guide_BSWE-pages-1-10.pdf"

my_processor = DocumentParser(DocumentConverter())
processor_result = my_processor.parse(source)

Converting document: assets/doc_storage/ECTS_Guide_BSWE-pages-1-10.pdf


In [4]:
class DocumentChunker:
    def __init__(self, tokenizer: str = "sentence-transformers/all-MiniLM-L6-v2"):
        self.tokenizer = tokenizer
        self.chunker = HybridChunker(tokenizer=tokenizer)

    def chunk(self, document, options: dict = None):
        """
        Splits the document into chunks.
        :param document: A DoclingDocument instance.
        :param options: Optional chunking parameters.
        :return: List of chunks ready for embedding.
        """
        print("Chunking document...")
        chunks = list(self.chunker.chunk(document))
        print(f"Created {len(chunks)} chunks")
        return chunks

In [5]:
my_chunker = DocumentChunker()
chunker_result = my_chunker.chunk(processor_result)

Token indices sequence length is longer than the specified maximum sequence length for this model (596 > 512). Running this sequence through the model will result in indexing errors


Chunking document...
Created 18 chunks


In [33]:
class DocumentEmbedder:
    def __init__(self, url, model_name):
        """
        :param url: The endpoint URL of the embedding service.
        :param model_name: The name of the model to use for generating embeddings.
        """
        self.url = url
        self.model_name = model_name

    def _post_request(self, texts):
        if isinstance(texts, str):
            texts = [texts]
        response = requests.post(
            self.url,
            json={
                "model": self.model_name,
                "input": texts
            }
        )
        response.raise_for_status()
        return response.json()

    def get_embeddings(self, texts):
        response = self._post_request(texts)
        embeddings = response["embeddings"]
        if isinstance(texts, str) or len(embeddings) == 1:
            return embeddings[0]
        return embeddings

    def get_full_response(self, texts):
        return self._post_request(texts)

    def embed(self, chunks):
        texts = [chunk.text for chunk in chunks]
        return self.get_embeddings(texts)
    
    def get_prepared_data_for_indexing(self, chunks):
        """
        Prepares data for indexing in a vector database.
        Each dictionary will contain 'vector', 'text', 'headings', and 'page_info' keys.
        :param chunks: A list of chunk objects, each having a 'text' attribute and optionally a 'meta' attribute.
        :return: List of dictionaries with embedding data.
        """
        embedding_result = self.embed(chunks)
        data = []
        for chunk, vector in zip(chunks, embedding_result):
            headings = ""
            page_info = ""
            if hasattr(chunk, "meta") and chunk.meta:
                # Use attribute access instead of .get() on the Pydantic model
                headings_list = getattr(chunk.meta, "headings", [])
                if headings_list:
                    headings = " > ".join(headings_list)
                page_info = getattr(chunk.meta, "page_info", "")
            data.append({
                "vector": vector,
                "text": chunk.text,
                "headings": headings,
                "page_info": page_info
            })
        return data

In [34]:
my_embedder = DocumentEmbedder(url='http://localhost:11434/api/embed', model_name="nomic-embed-text:latest")
embedding_result = my_embedder.embed(chunker_result)

In [35]:
print(len(embedding_result))
print(embedding_result[0])

18
[-0.026663823, 0.0011604077, -0.120403215, -0.034046873, 0.00759549, 0.018400041, 0.002070554, -0.017009337, -0.03305604, -0.017156463, -0.054219414, 0.047791604, 0.073238716, 0.039491843, 0.0020286015, 0.0057617207, -0.04884381, -0.031636912, -0.1274366, -0.036330443, 0.04193366, -0.01876864, -0.004979924, -0.06466832, 0.08445385, 0.09356817, -0.010465425, 0.001781235, -0.053709626, -0.059404925, 0.024018774, 0.017061926, 0.0140856, -0.053808954, -0.043438327, -0.050100863, 5.3711374e-06, -0.01964645, 0.007339668, -0.029310696, 0.025951087, -0.047145564, 0.0031650416, -0.053870477, 0.05945681, -0.022200206, 0.04960492, -0.025610838, 0.12549871, -0.018033413, -0.03257653, 0.03317217, 0.029279873, 0.014973368, 0.034225978, -0.00647258, -0.034419373, 0.036675677, -0.019148, -0.09079464, 0.033484735, 0.1201505, -0.060981225, 0.06364275, 0.024087291, -0.03581232, -0.028928682, 0.022279117, -0.008976748, 0.017833091, -0.0021952603, -0.033259388, 0.00071515574, -0.004746184, 0.0069233836,

In [36]:
prepared_embeddings = my_embedder.get_prepared_data_for_indexing(chunker_result)

In [37]:
print(prepared_embeddings[0])

{'vector': [-0.026663823, 0.0011604077, -0.120403215, -0.034046873, 0.00759549, 0.018400041, 0.002070554, -0.017009337, -0.03305604, -0.017156463, -0.054219414, 0.047791604, 0.073238716, 0.039491843, 0.0020286015, 0.0057617207, -0.04884381, -0.031636912, -0.1274366, -0.036330443, 0.04193366, -0.01876864, -0.004979924, -0.06466832, 0.08445385, 0.09356817, -0.010465425, 0.001781235, -0.053709626, -0.059404925, 0.024018774, 0.017061926, 0.0140856, -0.053808954, -0.043438327, -0.050100863, 5.3711374e-06, -0.01964645, 0.007339668, -0.029310696, 0.025951087, -0.047145564, 0.0031650416, -0.053870477, 0.05945681, -0.022200206, 0.04960492, -0.025610838, 0.12549871, -0.018033413, -0.03257653, 0.03317217, 0.029279873, 0.014973368, 0.034225978, -0.00647258, -0.034419373, 0.036675677, -0.019148, -0.09079464, 0.033484735, 0.1201505, -0.060981225, 0.06364275, 0.024087291, -0.03581232, -0.028928682, 0.022279117, -0.008976748, 0.017833091, -0.0021952603, -0.033259388, 0.00071515574, -0.004746184, 0.006

In [38]:
class VectorDBCreator:
    def __init__(self, milvus_client_name):
        self.milvus_client = MilvusClient(f"{milvus_client_name}.db")

    def create_collection(self, collection_name: str, dimension: int, **kwargs):
        if self.milvus_client.has_collection(collection_name=collection_name):
            self.milvus_client.drop_collection(collection_name=collection_name)
        self.milvus_client.create_collection(
            collection_name=collection_name,
            dimension=dimension,
            primary_field_name='id',
            id_type=DataType.INT64,
            vector_field_name='vector',
            extra_fields=[
                {"name": "text", "type": DataType.VARCHAR, "max_length": 1024},
                {"name": "headings", "type": DataType.VARCHAR, "max_length": 512},
                {"name": "page_info", "type": DataType.VARCHAR, "max_length": 128}
            ],
            metric_type='IP',  # Using Inner Product Distance
            auto_id=True,
            consistency_level='Strong',
            **kwargs
        )
        print(f"Collection '{collection_name}' with dimension {dimension} created.")
    
    def get_milvus_client(self):
        return self.milvus_client

In [39]:
my_vdb_creator = VectorDBCreator("my_vector_db_05")
my_vdb_creator.create_collection("my_collection", dimension=768)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Collection 'my_collection' with dimension 768 created.


In [40]:
class DocumentIndexer:
    def __init__(self, milvus_client, collection_name: str):
        """
        :param milvus_client: A Milvus client instance.
        :param collection_name: Name of the collection where vectors will be stored.
        """
        self.milvus_client = milvus_client
        self.collection_name = collection_name

    def index(self, data: list) -> dict:
        """
        Inserts data into the vector database.
        :param data: List of dicts, each with at least keys like 'vector' and 'text'.
        :return: Dict with indexing statistics.
        """
        print(f"Inserting {len(data)} vectors into collection '{self.collection_name}'...")
        self.milvus_client.insert(collection_name=self.collection_name, data=data)
        stats = {"indexed_count": len(data)}
        print(f"Finished indexing: {stats['indexed_count']} vectors inserted.")
        return stats

In [41]:
my_document_indexer = DocumentIndexer(my_vdb_creator.get_milvus_client(), "my_collection")
my_document_indexer.index(prepared_embeddings)

Inserting 18 vectors into collection 'my_collection'...
Finished indexing: 18 vectors inserted.


{'indexed_count': 18}

## Retriever

In [47]:
class DocumentRetriever:
    def __init__(self, milvus_client, collection_name: str, embedder):
        """
        :param milvus_client: A Milvus client instance.
        :param collection_name: Name of the collection to search.
        :param embedder: An embedding model (e.g., an instance of DocumentEmbedder that implements get_embeddings).
        """
        self.milvus_client = milvus_client
        self.collection_name = collection_name
        self.embedder = embedder

    def retrieve(self, query: str, k: int = 5) -> list:
        """
        Retrieve the top k most similar document chunks based on the query.
        :param query: Query string.
        :param k: Number of results to return.
        :return: List of chunks with metadata.
        """
        print(f"Processing query: {query}")
        query_embedding = self.embedder.get_embeddings(query)
        # Retrieve additional fields including headings and page_info
        search_results = self.milvus_client.search(
            collection_name=self.collection_name,
            data=[query_embedding],
            limit=k,
            search_params={"metric_type": "IP", "params": {}},
            output_fields=["text", "headings", "page_info"]
        )
        results = []
        # Assuming search_results[0] contains the results for our query.
        for res in search_results[0]:
            entity = res["entity"]
            results.append({
                "text": entity.get("text", ""),
                "headings": entity.get("headings", []),
                "page_info": entity.get("page_info", None),
                "distance": res["distance"]
            })
        return results

    def format_context(self, chunks: list) -> str:
        """
        Formats the retrieved chunks into a readable context string including headings.
        :param chunks: List of chunks with metadata.
        :return: Formatted context string.
        """
        context_parts = []
        for i, chunk in enumerate(chunks):
            headings_field = chunk.get("headings", None)
            if isinstance(headings_field, list):
                heading_path = " > ".join(headings_field) if headings_field else "Document section"
            elif isinstance(headings_field, str):
                heading_path = headings_field or "Document section"
            else:
                heading_path = "Document section"
                
            page_ref = f"(Page {chunk.get('page_info')})" if chunk.get('page_info') else ""
            context_parts.append(
                f"EXCERPT {i+1} - {heading_path} {page_ref}:\n{chunk['text']}\n"
            )
        return "\n".join(context_parts)

In [48]:
my_document_retriever = DocumentRetriever(my_vdb_creator.get_milvus_client(), "my_collection", my_embedder)

In [49]:
retriever_results = my_document_retriever.retrieve("Welche LV hat Tools für UI-Design?", k=3)

Processing query: Welche LV hat Tools für UI-Design?


In [50]:
show_pretty_json(retriever_results)

```json
[
  {
    "text": "LV Nummer Course number, 1 = I0859GPR03. LV Art Course Type, 1 = Integrierte Lehrveranstaltung Integrated Course. Semester, 1 = 3. Lehreinheiten Teaching units, 1 = 60. ECTS, 1 = 6 ECTS. Bewertungsmethode Evaluation method, 1 = Immanenter Pr\u00fcfungscharakter Continuous assessment. Lehrveranstaltungsinhalte Content, 1 = \u2022 Grundlagen des User Interface Designs \u2022 Kriterien f\u00fcr UI-Designs \u2022 Werkzeuge f\u00fcr die Oberfl\u00e4chengestaltung \u2022 Basics of user interface design \u2022 Criteria for UI designs \u2022 Tools for UI design\nLE0435_I_0859_ECTS_Guide_BSWE_2025 KP01LE - LE0400 erstellt: 29.04.2022/frn",
    "headings": "Human Interface Design",
    "page_info": "",
    "distance": 0.7386292219161987
  },
  {
    "text": "LV Art Course Type, I0859GDI02 = Integrierte Lehrveranstaltung Integrated Course. Semester, I0859GDI02 = 1. Lehreinheiten Teaching units, I0859GDI02 = 60. ECTS, I0859GDI02 = 6 ECTS. Bewertungsmethode Evaluation method, I0859GDI02 = Immanenter Pr\u00fcfungscharakter Continuous assessment. Lehrveranstaltungsinhalte Content, I0859GDI02 = \u2022 Systematik der Betriebssysteme \u2022 Speichersysteme, Cache und Speicherorganisation \u2022 E/A-Schnittstellen und Kommunikation \u2022 Interrupthandling \u2022 Pipelining \u2022 Superskalare und Multiprozessor-Architekturen \u2022 Sicherheitskonzepte in Betriebssystemen \u2022 Rechteverwaltung \u2022 Unix und Linux \u2022 Grundlagen Maschinencode \u2022 Windows \u2022 Bash und Powershell \u2022 Systematics of operating systems \u2022 Storage systems, cache, and storage organization \u2022 I/O interfaces and communication \u2022 Interrupt handling \u2022 Pipelining \u2022 Superscalar and multiprocessor architectures \u2022 Security concepts in operating systems \u2022 Rights management \u2022 Unix and Linux \u2022 Basics machine code \u2022 Windows \u2022 Bash and PowerShell",
    "headings": "Betriebssysteme / Operating Systems",
    "page_info": "",
    "distance": 0.6293371915817261
  },
  {
    "text": "LV Nummer Course number, 1 = I0859GDI04. LV Art Course Type, 1 = Integrierte Lehrveranstaltung Integrated Course. Semester, 1 = 2. Lehreinheiten Teaching units, 1 = 60. ECTS, 1 = 6 ECTS. Bewertungsmethode Evaluation method, 1 = Immanenter Pr\u00fcfungscharakter Continuous assessment. Lehrveranstaltungsinhalte Content, 1 = \u2022 LAN/WAN \u2022 OSI-Modell \u2022 TCP/IP Internet Layer \u2022 IP Adressierung und Subnetze \u2022 Transport Layer \u2022 Protokolle \u2022 Routing und Switching \u2022 Sicherheit in Netzwerken \u2022 LAN/WAN",
    "headings": "Grundlagen der Netzwerktechnologien / Foundations of Network Technologies",
    "page_info": "",
    "distance": 0.618161141872406
  }
]
```

In [53]:
formatted_context = my_document_retriever.format_context(retriever_results)
rprint(formatted_context)

EXCERPT 1 - Human Interface Design :
LV Nummer Course number, 1 = I0859GPR03. LV Art Course Type, 1 = Integrierte Lehrveranstaltung Integrated Course. 
Semester, 1 = 3. Lehreinheiten Teaching units, 1 = 60. ECTS, 1 = 6 ECTS. Bewertungsmethode Evaluation method, 1 = 
Immanenter Prüfungscharakter Continuous assessment. Lehrveranstaltungsinhalte Content, 1 = • Grundlagen des User 
Interface Designs • Kriterien für UI-Designs • Werkzeuge für die Oberflächengestaltung • Basics of user interface 
design • Criteria for UI designs • Tools for UI design
LE0435_I_0859_ECTS_Guide_BSWE_2025 KP01LE - LE0400 erstellt: 29.04.2022/frn

EXCERPT 2 - Betriebssysteme / Operating Systems :
LV Art Course Type, I0859GDI02 = Integrierte Lehrveranstaltung Integrated Course. Semester, I0859GDI02 = 1. 
Lehreinheiten Teaching units, I0859GDI02 = 60. ECTS, I0859GDI02 = 6 ECTS. Bewertungsmethode Evaluation method, 
I0859GDI02 = Immanenter Prüfungscharakter Continuous assessment. Lehrveranstaltungsinhalte Content, I0859GDI02 = • 
Systematik der Betriebssysteme • Speichersysteme, Cache und Speicherorganisation • E/A-Schnittstellen und 
Kommunikation • Interrupthandling • Pipelining • Superskalare und Multiprozessor-Architekturen • 
Sicherheitskonzepte in Betriebssystemen • Rechteverwaltung • Unix und Linux • Grundlagen Maschinencode • Windows • 
Bash und Powershell • Systematics of operating systems • Storage systems, cache, and storage organization • I/O 
interfaces and communication • Interrupt handling • Pipelining • Superscalar and multiprocessor architectures • 
Security concepts in operating systems • Rights management • Unix and Linux • Basics machine code • Windows • Bash 
and PowerShell

EXCERPT 3 - Grundlagen der Netzwerktechnologien / Foundations of Network Technologies :
LV Nummer Course number, 1 = I0859GDI04. LV Art Course Type, 1 = Integrierte Lehrveranstaltung Integrated Course. 
Semester, 1 = 2. Lehreinheiten Teaching units, 1 = 60. ECTS, 1 = 6 ECTS. Bewertungsmethode Evaluation method, 1 = 
Immanenter Prüfungscharakter Continuous assessment. Lehrveranstaltungsinhalte Content, 1 = • LAN/WAN • OSI-Modell •
TCP/IP Internet Layer • IP Adressierung und Subnetze • Transport Layer • Protokolle • Routing und Switching • 
Sicherheit in Netzwerken • LAN/WAN

## RAG-Tool mit Agent

In [88]:
# Configuration
class Config:
    OLLAPI_ENDPOINT_BASE = 'http://localhost:11434/v1'  # Base endpoint for Ollama
    OMODEL_LLAMA3D2 = 'llama3.2:latest'  # Model name

# Context class for agent state
class RAGContext(BaseModel):
    question: str = ""
    formatted_context: str = ""
    language: str = "English"  # Default language for responses

# Initialize the model - separate function for clarity
def create_llm_model():
    # Disable tracing to avoid messages about missing API keys
    set_tracing_disabled(True)
    
    # Create model with your endpoint
    return OpenAIChatCompletionsModel(
        model=Config.OMODEL_LLAMA3D2,
        openai_client=AsyncOpenAI(
            base_url=Config.OLLAPI_ENDPOINT_BASE, 
            api_key="fake-key"  # Using fake key as local endpoint doesn't require auth
        )
    )

# Knowledge retrieval tool that uses your DocumentRetriever
@function_tool
async def query_ects_guide(
    context: RunContextWrapper[RAGContext], 
    query: str, 
    k: int = 3
    ) -> str:
    """
    Query the ECTS guide knowledge base to retrieve relevant information.
    
    Args:
        query: The user's question about courses, ECTS, or study programs
        k: Number of top results to return (default: 3)
    
    Returns:
        Formatted context with relevant information from the ECTS guide
    """
    try:
        # Store the question in context
        context.context.question = query
        
        # Use your existing document retriever
        retriever_results = my_document_retriever.retrieve(query, k=k)
        formatted_context = my_document_retriever.format_context(retriever_results)
        
        # Store formatted context in the agent context
        context.context.formatted_context = formatted_context
        
        return formatted_context
    except Exception as e:
        error_msg = f"Error retrieving documents: {str(e)}"
        print(error_msg)
        return error_msg

# Create a single RAG agent - simplifying the design
# Using valid tool_use_behavior value
rag_agent = Agent[RAGContext](
    name="ECTS Guide Assistant",
    instructions="""
    You are an assistant specialized in helping students understand course information,
    ECTS credits, and study programs.
    
    WORKFLOW:
    1. When a user asks a question, use the query_ects_guide tool to retrieve relevant information
    2. Carefully analyze the retrieved context
    3. Provide a clear, accurate answer based on the retrieved information
    4. If the information isn't available in the retrieved context, indicate this clearly
    
    Respond in English.
    Be helpful, accurate, and concise in your responses.
    """,
    tools=[query_ects_guide],
    model=create_llm_model(),
    # Using a valid tool_use_behavior value
    tool_use_behavior="run_llm_again",
    # Set model settings to help with function calling
    model_settings=ModelSettings(
        temperature=0.1,  # Lower temperature for more deterministic responses
        tool_choice="auto"  # Auto tool choice
    )
)

# Manual process for RAG when the model doesn't handle function calling properly
async def manual_rag_process(question, k=1):
    """
    Manually execute the RAG process when the model doesn't properly use function calling.
    """
    try:
        # Direct call to retrieve documents
        retriever_results = my_document_retriever.retrieve(question, k=k)
        formatted_context = my_document_retriever.format_context(retriever_results)
        
        # Create a prompt with the retrieved context
        formatted_prompt = f"""
        Question: {question}

        Context from ECTS guide:
        {formatted_context}

        Based on the above context, please provide a concise and accurate answer to the question.
        """
        
        # Create context with the retrieved info
        context = RAGContext(
            question=question,
            formatted_context=formatted_context,
            language="English"
        )
        
        # Use a list for input items with the formatted prompt
        input_items = [{"content": formatted_prompt, "role": "user"}]
        
        # Run the model with this prompt
        run_result = await Runner.run(
            rag_agent,
            input=input_items,
            context=context
        )
        
        # Return the results
        return {
            "answer": run_result.final_output,
            "run_result": run_result,
            "context": context
        }
    except Exception as e:
        error_msg = f"Error in manual RAG process: {str(e)}"
        print(error_msg)
        return {"error": error_msg}

# Detailed trace function for better visibility into the agent process
def print_run_trace(run_result):
    """
    Prints a detailed, readable trace of the agent run process
    showing each step in the interaction.
    
    Args:
        run_result: The RunResult object from an agent run
    """
    print("\n== DETAILED AGENT RUN TRACE ==\n")
    
    # Print basic info
    print(f"Total items generated: {len(run_result.new_items)}")
    print(f"Model responses: {len(run_result.raw_responses)}")
    print("-" * 80)
    
    # Loop through each item and print details based on item type
    for i, item in enumerate(run_result.new_items):
        item_type = getattr(item, 'type', 'unknown')
        
        print(f"\n[STEP {i+1}: {item_type}]")
        
        if item_type == 'tool_call_item':
            # Print tool call details
            print(f"  Agent: {item.agent.name}")
            raw_item = item.raw_item
            print(f"  Tool called: {raw_item.name}")
            print(f"  Arguments: {raw_item.arguments}")
            
        elif item_type == 'tool_call_output_item':
            # Print tool output details
            print(f"  Agent: {item.agent.name}")
            print(f"  Output type: {item.raw_item.get('type', 'unknown')}")
            
            # Truncate long outputs for readability
            output = item.output
            if len(output) > 150:
                output = output[:150] + "..."
            print(f"  Output: {output}")
            
        elif item_type == 'message_output_item':
            # Print message details
            print(f"  Agent: {item.agent.name}")
            raw_item = item.raw_item
            
            # Extract and format content
            content = ""
            if hasattr(raw_item, 'content') and raw_item.content:
                for content_item in raw_item.content:
                    if hasattr(content_item, 'text'):
                        text = content_item.text
                        if len(text) > 150:
                            text = text[:150] + "..."
                        content = text
            
            print(f"  Role: {getattr(raw_item, 'role', 'unknown')}")
            print(f"  Content: {content}")
            
        else:
            # For any other item types
            print(f"  Item details: {item}")
            
        print("-" * 80)
    
    # Print token usage information if available
    print("\n== TOKEN USAGE ==")
    total_input_tokens = 0
    total_output_tokens = 0
    
    for i, response in enumerate(run_result.raw_responses):
        if hasattr(response, 'usage'):
            usage = response.usage
            input_tokens = getattr(usage, 'input_tokens', 0)
            output_tokens = getattr(usage, 'output_tokens', 0)
            total_tokens = getattr(usage, 'total_tokens', 0)
            
            print(f"Response {i+1}:")
            print(f"  Input tokens: {input_tokens}")
            print(f"  Output tokens: {output_tokens}")
            print(f"  Total tokens: {total_tokens}")
            
            total_input_tokens += input_tokens
            total_output_tokens += output_tokens
    
    print(f"\nTotal input tokens: {total_input_tokens}")
    print(f"Total output tokens: {total_output_tokens}")
    print(f"Grand total tokens: {total_input_tokens + total_output_tokens}")

# Main function with detection of function calling issues
async def ask_about_ects(
    question: str, 
    document_retriever=None, 
    language="English"
    ) -> Dict:
    """
    Process a question through the ECTS guide RAG pipeline with auto-detection
    of function calling issues.
    
    Args:
        question: The user's question about the ECTS guide
        document_retriever: An instance of DocumentRetriever (optional if already defined globally)
        language: Response language (default: English)
    
    Returns:
        Dictionary containing answer, run_result, and context for inspection
    """
    # Allow passing a document retriever if not defined globally
    global my_document_retriever
    if document_retriever is not None:
        my_document_retriever = document_retriever
    
    # Create context with specified language
    context = RAGContext(language=language)
    
    # First attempt: standard approach
    with trace("ECTS Guide Query - Standard Approach"):
        # Use a list for input items
        input_items = [{"content": question, "role": "user"}]
        
        # Run the agent
        run_result = await Runner.run(
            rag_agent,
            input=input_items,
            context=context
        )
    
    # Check if we got a proper answer or just a function call spec
    is_function_call_text = False
    if run_result.final_output:
        # Check if the output looks like a raw function call
        if run_result.final_output.startswith('{"name":') or \
           'query_ects_guide' in run_result.final_output:
            is_function_call_text = True
    
    # If the model returned a function call as text, use manual RAG approach
    if is_function_call_text:
        print("Detected function call specification in output. Switching to manual RAG process...")
        return await manual_rag_process(question, k=1)
    
    # Standard approach worked fine
    return {
        "answer": run_result.final_output,
        "run_result": run_result,
        "context": context
    }

# Example usage
async def demo_rag():
    #question = "Which course type is Foundations of Network Technologies?"
    question = "Will I/O interfaces and communication be covered in the course Operating Systems?"
    result = await ask_about_ects(question, my_document_retriever)
    
    # Print the answer
    print("ANSWER:")
    print(result["answer"])
    
    # Use our detailed trace function
    print_run_trace(result['run_result'])
    
    # Print context preview
    context = result.get('context')
    if context and hasattr(context, 'formatted_context'):
        print("\n== RETRIEVED CONTEXT ==")
        print(context.formatted_context)
    
    return 'sucess' #result

# For Jupyter notebook execution
await demo_rag()

Processing query: Operating Systems course I/O interfaces and communication
ANSWER:
Based on the retrieved information from the ECTS guide, it appears that I/O interfaces and communication will be covered in the course "Operating Systems". Specifically, Excerpt 1 mentions "E/A-Schnittstellen und Kommunikation" which translates to "I/O interfaces and communication" as part of the course content.

Therefore, yes, I/O interfaces and communication will be covered in the course Operating Systems.

== DETAILED AGENT RUN TRACE ==

Total items generated: 3
Model responses: 2
--------------------------------------------------------------------------------

[STEP 1: tool_call_item]
  Agent: ECTS Guide Assistant
  Tool called: query_ects_guide
  Arguments: {"k":"2","query":"Operating Systems course I/O interfaces and communication"}
--------------------------------------------------------------------------------

[STEP 2: tool_call_output_item]
  Agent: ECTS Guide Assistant
  Output type: functio

'sucess'